## Introduction: Map Area

In this project, we analyze a map of a segment of Los Angeles, California. Specifically, I've decided to focus on an area of LA encompassing Downtown Los Angeles (DTLA) and the immediate area of the University of Southern California (USC).

Having attended USC from 2010-2015, I spent the majority of my time during those five years within a 5-10 mile radius of campus. While I think I know the area somewhat well, I figured it would be interesting to analyze a dataset that would allow me to uncover new insights on a portion of Los Angeles that I feel a strong personal connection with.

I used Mapzen's - an open source mapping platform -  custom extract tool to create my map. 

- Map Extract (USC and Downtown LA Area): https://mapzen.com/data/metro-extracts/your-extracts/0addcca0d637

Additionally, I used LA Times' profile on Downtown LA to find out the boundaries of the area. I tried to include as much of DTLA I could during my map extraction, however the limitations of the tool meant I couldn't grab 100% of the area.

- LA Times Profile on Downtown LA: http://maps.latimes.com/neighborhoods/neighborhood/downtown/

Here's the map of the combined USC and DTLA area that I used for analysis:

In [1]:
from IPython.display import Image
Image(url='http://i.imgur.com/XX9Aq7z.png')

## Data Cleaning

Before analyzing the map, I decided to take a look at the dataset (a 121 MB .OSM OpenStreetMap file coded in XML) and see if data cleaning would be necessary. 

### Problems Encountered in the Dataset

Looking at the dataset, there were a few issues I encountered. I knew I'd have to engage in some data wrangling to fix the following problems:

1) Street Names - Abbreviations of street types ("Figueroa St"), incomplete street names ("S. San Pedro"), and street names with too much information ("Traction Avenue, Suite 3A").

2) Post Codes - Post codes with nine digits instead of five ("90007-2030"), incomplete post codes ("9006", typo for "90006"), and erroneously entered post codes ("CA", "90007-2030 213-749-9646", "CA 90018").

3) Phone Numbers - General lack of uniformity among phone numbers. 

Next, we'll tackle these issues in that order.

### Fixing Street Names

The ideal street name is a complete street name, with no abbreviations for the street type, and no excess information. 

For example:

- *Figueroa St* and *Olympic Boulevard, Unit 112*

Should be corrected to:

- *Figueroa Street* and *Olympic Boulevard*



I identified the problem street names via a few steps. The *re* (regular expression) module was used to search the end of street names to find the street type ("St", "Street", "Avenue", etc). The specific regular expression used was as follows:

In [ ]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

m = street_type_re.search(street_name)

Once the street type was found, the street type was matched against a list of "expected" street types. If the street type was not in that expected list, the entire street name would be added to a dictionary reserved for street names with problem street types, where the key of the dictionary is the problem street type (like "St") and its value is a list of street names that contain that problem street type (such as ["Figueroa St", "Third St"]).

Once I knew what problem street types existed, I created the audit code to fix them. Key to my audit code are my "mapping" and "street_correction" dictionaries, that replace all of the problem street types, and some particularly problematic street names, with their corrected versions.

For streets with either wrong or incomplete information, I engaged in outside research to find out the necessary information for fixes. 

Here's a snippet of the code I used to clean the street names:

In [2]:
# Function to Update Street Names
def update_name(name, mapping, street_correction):
    m = street_type_re.search(name) # Recompiler is used to look for a match. Helps us find the street type.
    street_type = m.group() # This allows us to work with the street type we grabbed
    if street_type not in expected: # This ensures it only operates update_names on street names that need to be updated
        if street_type not in mapping: 
            name = street_correction[street_type] # This is to correct the ones who whose entire name need to be corrected
        else:
            corrected_street_type = mapping[street_type] # Gets us the corrected street type from the mapping dictionary
            name = name.replace(street_type, corrected_street_type) # Replaces the instance of the incorrected street type 
    return name # Return the corrected street name

### Fixing Phone Numbers

Phone numbers have been inputted a variety of ways.

The format I've decided that all phone numbers should be in is the following: 

- 012-345-6789

Any number that isn't in that format (has a country code, parentheses around area codes, no spaces, etc) will be corrected. 

For example:

- *+1.855.427.6547*,  *+12136878808* and *+1 (213) 748-4141*

Should be corrected to:

- *855-427-6547*, *213-687-8808* and *213-748-4141*

I identified all of the problem phone numbers by utilizing a function that would take in two arguments: an element from the XML data, and a set reserved for all of the phone numbers. It looked through the element for a phone number, and if it was found, it added that phone number to the phone numbers set.

A quick look through the phone numbers allowed me to notice how inconsistent phone number formatting was.

Here's the code I used to standardize the phone numbers:

In [3]:
def update_phone_number(phone_number):
    phone_number = re.sub("[-+(). ]", "", phone_number) # Through the regular expression module, replace any instances
    # of those symbols with "" - essentially removes those symbols from the phone numbers
    if len(phone_number) > 10: # If the number has more than 10 numbers, it has a country code
        phone_number = phone_number[1:] # Delete the country code
    phone_number = phone_number[0:3] + "-" + phone_number[3:6] + "-" + phone_number[6:] # This converts the numbers into the 
    # 012-345-678 format
    return phone_number # Return the standarized phone number

### Fixing Post Codes

Post codes should be standardized to a five digit format e.g. 90007.

Some post codes have an extra four digits that identify a more specific area within the post code. I want to keep our
post codes consistent, so I removed the extra four digits from our data.

Others simply have input errors.

For example:

- *CA 90012*, *90057-4101* and *90007-2030 213-749-9646*

Should be corrected to:

- *90012*, *90057* and *90007*

I identified all of the problem post codes by creating a function that would take in two arguments: an element from the XML data, and a set reserved for all of the post codes. It searched the element for a post code, and then added that post code to the set of post codes.

Similar to the phone numbers, I noticed there wasn't a consistent format for all of the post codes.

Here's the code I used to standardize them:

In [ ]:
def update_post_code(post_code):
    if post_code == 'CA': # This is to fix the post code that just has 'CA' as its post code. 
        # It is corrected to the location's actual post code.
        post_code = '90057'
    elif post_code == '9006': # This is to fix the post code that just has '9006" as it's post code. 
        #It is corrected to the actual post code.
        post_code = '90006'
    else:
        index = post_code.find('9') # Grabs the index of the first instance of the number 9 in any given post code
        post_code = post_code[index:(index+5)] # Utilize index slicing to obtain a post code that starts with 9 and 
        # contains the following four digits
    return post_code # Return the standardized post code

## Parsing XML and Importing to CSV

Next, I used Python code that parses the map dataset, utilizing the aforementioned functions in order to clean the phone numbers, post codes and street names, and then imports the entire dataset into five CSV files. 

The actual code used for the auditing and importing is quite long, but here's a snippet of the *shape_element* function, where you can see how the functions that we defined earlier (*update_name*, *update_phone_number*, *update_post_code*) are used to audit the XML file. This portion of the shape_element function focuses on updating ways tags and nodes tags.

In [ ]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    # Filling up the tags list:
    for tag in element.iter("tag"):
        dic = {}
        dic['id'] = element.attrib['id']
        if ":" in tag.attrib['k']: # Checks if there's a colon in the k value like "addr:street"
            splitkey = re.split(":",tag.attrib["k"],1) # This allows us to split on the colon but only the first one
            dic['key'] = splitkey[1] 
            if dic['key'] == 'street':
                dic['value'] = update_name(tag.attrib["v"], mapping, street_correction) # Updates street name
            elif dic['key'] == 'postcode':
                dic['value'] = update_post_code(tag.attrib["v"]) # Updates post code
            elif dic['key'] == 'phone':
                dic['value'] = update_phone_number(tag.attrib["v"]) # Updates phone number
            else:
                dic['value'] = tag.attrib['v']
            dic['type'] = splitkey[0]
            
        ...   
        
        tags.append(dic)

## Importing CSV Files into SQLite Database#

In order to perform analysis on my data using Python and SQL, I had to convert my new CSV files into SQL tables.

My SQL program of choice is SQLite. I used sqlite3, accessed via my PC's command line, to make a new database, create the five SQL tables with specific schemas, and load the CSVs into each of them.

Once the tables were created, the relevant CSV files were imported to each SQL table (e.g. 'nodes.csv' was imported to the table 'nodes')

## Function for Retrieving Data from SQLite

We will create a function, titled *sqloutput*, that will take a SQL query as it's input and output the results into a dataframe.

When the function obtains the SQL query, it will then establish the connection to our database, *udacityp3.db*, using the *sqlite3* module.

Next, using the read_sql_query function from the *pandas* module, it will process the query and pass it into a pandas dataframe.

The connection will then be closed, and the function will return the dataframe of our results.

In [4]:
import pandas as pd
import sqlite3

def sqloutput(query):
    conn = sqlite3.connect("udacityp3.db") #Establish connection to database
    df = pd.read_sql_query(query, conn) # Execute the query and pass results into a dataframe
    conn.close() # Close the connection
    return df # Return the query results

## Data Overview

For now, let's understand out data on a broad level by looking at the number of nodes and ways in the dataset, and the number of users who've contributed to this portion of the Los Angeles map.

We'll be using SQL queries throughout this project to obtain the desired data.

### File Sizes

    USC-and-Downtown-Los-Angeles.osm ......... 121 MB
    udacityp3.db ............................. 111 MB
    nodes.csv ............................... 48.8 MB
    nodes_tags.csv .......................... 0.40 MB
    ways.csv ................................. 3.7 MB
    ways_tags.csv ........................... 12.3 MB
    ways_nodes.csv .......................... 14.9 MB  

### Number of Nodes

In [5]:
sqloutput("SELECT COUNT(*) FROM nodes")

,COUNT(*)
0,532044


### Number of Ways

In [6]:
sqloutput("SELECT COUNT(*) FROM ways")

,COUNT(*)
0,56780


### Number of Users Who've Contributed to this Map

In [12]:
sqloutput("SELECT COUNT(*) FROM (SELECT user FROM ways UNION SELECT user FROM nodes)")

,COUNT(*)
0,419


### Top 10 Contributing Users

In [47]:
sqloutput("SELECT user, COUNT(*) as Contributions FROM (SELECT user FROM ways UNION ALL SELECT user FROM nodes) GROUP BY user ORDER BY Contributions DESC LIMIT 10")

,user,Contributions
0,schleuss_imports,191201
1,joemfox_imports,87066
2,dannykath_labuildings,55268
3,karitotp_labuildings,36399
4,RichRico_labuildings,35698
5,dima_import,26209
6,samely_labuildings,17743
7,emamd_imports,17359
8,calfarome_labuilding,11105
9,piligab_labuildings,7838


### Numbers of Users Who Have Only Contributed Once

In [48]:
sqloutput("SELECT COUNT(*) FROM (SELECT user, COUNT(*) as Contributions FROM (SELECT user FROM ways UNION ALL SELECT user FROM nodes) GROUP BY user HAVING Contributions = 1)")

,COUNT(*)
0,96


## Additional Analysis

Now that we've gotten an overview, let's do some additional analysis on the dataset. 

### Top 10 Type of Restaurants

In [49]:
sqloutput("SELECT value, count(*) AS num FROM nodes_tags WHERE key = 'cuisine' GROUP BY value ORDER BY num DESC LIMIT 10 ")

,value,num
0,japanese,14
1,american,9
2,coffee_shop,9
3,sandwich,9
4,mexican,8
5,pizza,8
6,burger,4
7,korean,3
8,chicken,2
9,chinese,2


### Top Places of Worship

In [50]:
sqloutput("SELECT value, count(*) AS num FROM nodes_tags WHERE key = 'religion' GROUP BY value ORDER BY num DESC LIMIT 10 ")

,value,num
0,christian,86
1,buddhist,3
2,catholic,1


### Top 10 Amenities

In [51]:
sqloutput("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key='amenity' GROUP BY value ORDER BY num DESC LIMIT 10")

,value,num
0,restaurant,126
1,place_of_worship,97
2,bicycle_rental,51
3,school,46
4,fast_food,28
5,cafe,24
6,bench,22
7,fountain,22
8,parking,22
9,bicycle_parking,19


## Potential Improvements

The biggest issue OpenStreetMap (OSM) has is the relatively little data they have. A map application lives and dies by it's data, and it's fair to assume that individuals will immediately seek out alternatives if they can't find what they're looking for on OpenStreetMap. There's simply not enough people adding to the map. OpenStreet Map has to get more people on their platform.


### Increase Engagement

In order to get more users to populate OpenStreetMap with information, OpenStreetMap should collaborate with app developers to use OpenStreetMap as their main mapping software. Specifically, apps that have features like geo-tagging and check-ins have the potential to really boost OSM. The data they input into these apps could then be inputted into OpenStreetMap. 

#### Potential Obstacle

However, considering that OSM is a non-profit, and not a profit making corporation, they may struggle convincing a significant amount of developers to integrate OSM into their apps.

### Stronger Data Input Guidelines

Create stronger guidelines that will help keep information consistent throughout. For instance, "fast_food" has been inputted as an "amenity" by some users, and as a "cuisine" by others. This can make trying to find the data you want slightly trickier than you would like. Clearer guidelines that layout where exactly some pieces of information should go will help keep the data consistent. 

#### Potential Obstacle

This is easier said than done, however, as there are many different kinds of data and possible inputs that need to be taken into consideration when writing these guidelines. This could make guidelines that are too long to read. Also, OpenStreetMap may struggle getting a large amount of their users to thoroughly read the guidelines.

## Conclusion

The map containing the USC and DTLA area doesn't have as much information as I hoped. The consequence of not having enough information means that I might not have much confidence toward my data analysis findings if I were to conduct a deep investigation, or I just won't find the information I need. 

Hopefully, as OpenStreetMap grows, and more than just 419 people add to the map of this part of Los Angeles, OSM will not only have more information, but potentially better organized data as well. This will help users like myself who wish to learn more about cities by analyzing a map dataset.